# Batch Data Processing using Spark on Databricks

Apache Spark is a powerful open-source distributed computing system that provides fast and general-purpose cluster computing for big data processing.

Databricks, on the other hand, is a cloud-based platform built on top of Apache Spark, making it easier to deploy and manage Spark clusters. 
Databricks provides a unified analytics platform that can process large amounts of data quickly. 
Databricks provides an optimised and managed Spark environment.

To clean and query the data from the three Kafka topics, the S3 bucket will be mounted to a Databricks account. Within Databricks three DataFrames will be created to hold this data:

- `df_pin` for the Pinterest post data
- `df_geo` for the geolocation data
- `df_user` for the user data.

This notebook will falcitate the following procedures:

- Load the batch data
- Clean the batch data
- Query the batch data

This notebook uses the ***mounting*** methods and dataframe creation methods from the `databricks_load_data` notebook located in the `classes` folder.

This notebook also uses the dataframe cleaning methods from the `databricks_clean_data` file also located in the `classes` folder.

#### Import loading methods 

The following cell allows access to the methods from the `S3DataLoader` class within the `databricks_load_data` notebook.

In [0]:
%run "./classes/databricks_load_data"

#### Import cleaning methods

The following cell allows access to the methods from the `DataCleaning` class within the `databricks_clean_data` notebook

In [0]:
%run "./classes/databricks_clean_data"

#### Instantiate S3DataLoader and DataCleaning

The following cell instantiates the required variables for the `S3DataLoader` class and `DataCleaning` class.

In [0]:
if __name__ == "__main__":
    credentials_path = "dbfs:/user/hive/warehouse/authentication_credentials"
    iam_username = "0ab336d6fcf7"
    topics = ['pin', 'geo', 'user']
    data_loader = S3DataLoader(credentials_path, iam_username, topics)
    data_cleaner = DataCleaning()

## Load Batch Data

Databricks enables users to mount cloud object storage to the Databricks File System (DBFS) to simplify data access patterns.

Databricks mounts create a link between a workspace and cloud object storage, which enables interaction with cloud object storage using familiar file paths relative to the Databricks file system. Mounts work by creating a local alias under the /mnt directory that stores the following information:

- Location of the cloud object storage.
- Driver specifications to connect to the storage account or container.
- Security credentials required to access the data.

#### Mount S3 bucket

As per the project instructions the following cell mounts an external S3 bucket to Databricks Filesystem.

In [0]:
if __name__ == "__main__":
    # Load AWS keys and configure S3 bucket
    data_loader.mount_s3_bucket()

#### Show file structure

The following cell will show the file structure within the mount path.

In [0]:
if __name__ == "__main__":
    # Display the mounted S3 bucket
    data_loader.display_s3_bucket()

#### Create dataframes

The following cell will create three dataframes from the data stored in the mount path.

In [0]:
if __name__ == "__main__":
    # Create dataframes
    data_loader.create_dataframes(mounted=True)

## Clean Batch Data

#### Clean df.pin

To clean the `df_pin` DataFrame the following cell will perform the following transformations:

- Replace empty entries and entries with no relevant data in each column with `Nones`
- Perform the necessary transformations on the `follower_count` to ensure every entry is a number. Make sure the data type of this column is an `int`.
- Ensure that each column containing numeric data has a `numeric` data type
- Clean the data in the `save_location` column to include only the save location path
- Rename the `index` column to `ind`.
- Reorder the DataFrame columns to have the following column order: (`ind`, `unique_id`, `title`, `description`, `follower_count`, `poster_name`, `tag_list`, `is_image_or_video`, `image_src`, `save_location`, `category`) 

In [0]:
if __name__ == "__main__":
    cleaned_df_pin = data_cleaner.clean_pin_data(df_pin)

    print("Schema for original dataframe")
    df_pin.printSchema()
    print("Schema for cleaned dataframe")
    cleaned_df_pin.printSchema()

    print("Original dataframe:")
    display(df_pin)
    print("Cleaned dataframe:")
    display(cleaned_df_pin)

#### Clean df.geo

To clean the `df_geo` DataFrame the follwoing cell will perform the following transformations:

- Create a new column `coordinates` that contains an array based on the `latitude` and `longitude` columns
- Drop the `latitude` and `longitude` columns from the DataFrame
- Convert the `timestamp` column from a `string` to a `timestamp` data type
- Reorder the DataFrame columns to have the following column order: (`ind`, `country`, `coordinates`, `timestamp`)


In [0]:
if __name__ == "__main__":
    cleaned_df_geo = data_cleaner.clean_geo_data(df_geo)

    print("Schema for original dataframe")
    df_geo.printSchema()
    print("Schema for cleaned dataframe")
    cleaned_df_geo.printSchema()

    print("Original dataframe:")
    display(df_geo)
    print("Cleaned dataframe:")
    display(cleaned_df_geo)

#### Clean df.user

To clean the `df_user` DataFrame the following cell will perform the following transformations:

- Create a new column user_name that concatenates the information found in the `first_name` and `last_name` columns
- Drop the `first_name` and `last_name` columns from the DataFrame
- Convert the `date_joined` column from a `string` to a `timestamp` data type
- Reorder the DataFrame columns to have the following column order: (`ind`, `user_name`, `age`, `date_joined`)

In [0]:
if __name__ == "__main__":
    cleaned_df_user = data_cleaner.clean_user_data(df_user)

    print("Schema for original dataframe")
    df_user.printSchema()
    print("Schema for cleaned dataframe")
    cleaned_df_user.printSchema()
    
    print("Original dataframe:")
    display(df_user)
    print("Cleaned dataframe:")
    display(cleaned_df_user)

## Query Batch Data

Before querieing the data the three dataframes (`df_pin`, `df_geo`, and `df_user`) are joined together on the common column heading `ind`.

To make sure that `df_all` is a valid DataFrame it will be created and registered as a temporary table or view before executing any SQL queries. To do this `df_all` is registered as a temporary view using `df_all.createOrReplaceTempView("df_all")`.

However `df_all` is a non-Delta table with many small files. Therefore to improve the performance of queries, `df_all` has been converted to Delta. The new `df_all` table will accelerate queries.

In [0]:
if __name__ == "__main__":
    # Join the three dataframes
    df_all = cleaned_df_pin.join(cleaned_df_geo, 'ind').join(cleaned_df_user, 'ind')

    # Write the Delta table
    delta_table_path = "/delta/my_table/mount"
    df_all.write.format("delta").mode("overwrite").save(delta_table_path)

    # Use DeltaTable API to optimize the Delta table
    delta_table = DeltaTable.forPath(spark, delta_table_path)
    delta_table.vacuum()
    df_all = delta_table.toDF()

    # Create a temperory table of the combined dataframes
    df_all.createOrReplaceTempView("df_all")

    # Display the optimized Delta tables
    display(df_all)

#### Question 1: Find the most popular category in each country

- Find the most popular Pinterest category people post to based on their country.
- The query should return a DataFrame that contains the following columns: (`country`, `category`, `category_count`)


In [0]:
top_category_per_country = spark.sql("""
    WITH ranked_categories AS (
        SELECT
            country, 
            category, 
            COUNT(category) AS category_count, 
            RANK() OVER (PARTITION BY country ORDER BY count(category) DESC) AS category_rank 
        FROM df_all 
        GROUP BY country, category 
    ) 
    SELECT country, category, category_count 
    FROM ranked_categories 
    WHERE category_rank = 1 
    ORDER BY country
""")
display(top_category_per_country)

#### Question 2: Find which was the most popular category each year

- Find how many posts each category had between 2018 and 2022.
- The query will return a DataFrame that contains the following columns: (`post_year`, `category`, `category_count`)

In [0]:
top_category_per_year = spark.sql("""
    WITH ranked_categories AS (
        SELECT
            category,
            EXTRACT(YEAR FROM timestamp) AS post_year,
            COUNT(category) AS category_count,
            RANK() OVER (PARTITION BY EXTRACT(YEAR FROM timestamp) ORDER BY count(category) DESC) AS category_rank 
        FROM df_all 
        GROUP BY EXTRACT(YEAR FROM timestamp), category
    )
    SELECT post_year, category, category_count 
    FROM ranked_categories 
    WHERE category_rank = 1 AND post_year BETWEEN 2018 AND 2022
    ORDER BY post_year
""")
display(top_category_per_year)

#### Question 3: Find the user with most followers in each country

- Step 1: For each country find the user with the most followers.
  - Your query should return a DataFrame that contains the following columns: (`country`, `poster_name`, `follower_count`)
- Step 2: Based on the above query, find the country with the user with most followers.
  - Your query should return a DataFrame that contains the following columns: (`country`, `follower_count`)
  - This DataFrame should have only one entry.

In [0]:
# Step 1:
top_user_per_country = spark.sql("""
    SELECT country, poster_name, MAX(follower_count) AS follower_count
    FROM df_all
    GROUP BY country, poster_name
    ORDER BY follower_count DESC
""")
display(top_user_per_country)

# Step 2:
country_with_top_user = spark.sql("""
    WITH top_users AS (
        SELECT country, poster_name, MAX(follower_count) AS follower_count
        FROM df_all
        GROUP BY country, poster_name
        ORDER BY follower_count DESC
    )
    SELECT country, follower_count
    FROM top_users
    ORDER BY follower_count DESC
    LIMIT 1
""")
display(country_with_top_user)


#### Question 4: Find the most popular category for different age groups

- What is the most popular category people post to based on the following age groups: (`18-24`, `25-35`, `36-50`, `+50`)
- The query should return a DataFrame that contains the following columns: (`age_group`, `category`, `category_count`)

In [0]:
top_category_per_age = spark.sql("""
    WITH age_groups AS (
        SELECT 
            CASE 
                WHEN Age BETWEEN 18 AND 24 THEN '18-24'
                WHEN Age BETWEEN 25 AND 35 THEN '25-35'
                WHEN Age BETWEEN 36 AND 50 THEN '36-50'
                WHEN Age >= 51 THEN '50+'
                ELSE 'NA'
            END AS age_group,
            category, 
            COUNT(category) AS category_count
        FROM df_all
        GROUP BY age_group, category
    ),
    ranked_ages AS (
        SELECT 
            age_group, 
            category, 
            category_count, 
            RANK() OVER (PARTITION BY age_group ORDER BY category_count DESC) AS category_rank
        FROM age_groups
    )
    SELECT age_group, category, category_count
    FROM ranked_ages
    WHERE category_rank = 1
""")
display(top_category_per_age)


#### Question 5: Find the median follower count for different age groups

- What is the median follower count for users in the following age groups: (`18-24`, `25-35`, `36-50`, `+50`)
- The query should return a DataFrame that contains the following columns: (`age_group`, `median_follower_count`)


In [0]:
median_follower_per_age_group = spark.sql("""
    SELECT 
        CASE 
            WHEN Age BETWEEN 18 AND 24 THEN '18-24'
            WHEN Age BETWEEN 25 AND 35 THEN '25-35'
            WHEN Age BETWEEN 36 AND 50 THEN '36-50'
            WHEN Age >= 51 then '50+'
            ELSE 'NA'
        End as age_group,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count 
    FROM df_all
    GROUP BY age_group
    ORDER BY median_follower_count DESC
""")
display(median_follower_per_age_group)                                                                

#### Question 6: Find how many users have joined each year?

- Find how many users have joined between 2015 and 2020.
- The query should return a DataFrame that contains the following columns: (`post_year`, `number_users_joined`)

In [0]:
users_joined_per_year = spark.sql("""
    SELECT
        EXTRACT(YEAR FROM date_joined) AS post_year,
        COUNT(DISTINCT(user_name)) AS number_users_joined
    FROM df_all 
    WHERE EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
    GROUP BY post_year
""")
display(users_joined_per_year)

#### Question 7: Find the median follower count of users based on their joining year

- Find the median follower count of users have joined between 2015 and 2020.
- Your query should return a DataFrame that contains the following columns: (`post_year`, `median_follower_count`)

In [0]:
median_follower_per_join_year = spark.sql("""
    SELECT
        EXTRACT(YEAR FROM date_joined) AS post_year,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
    FROM df_all 
    WHERE EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
    GROUP BY post_year
""")
display(median_follower_per_join_year)

#### Question 8: Find the median follower count of users based on their joining year and age group

- Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
- The query should return a DataFrame that contains the following columns: (`age_group`, `post_year`, `median_follower_count`)

In [0]:
median_follower_per_join_year_age_group = spark.sql("""
    SELECT
        CASE 
            WHEN Age BETWEEN 18 AND 24 THEN '18-24'
            WHEN Age BETWEEN 25 AND 35 THEN '25-35'
            WHEN Age BETWEEN 36 AND 50 THEN '36-50'
            WHEN Age >= 51 then '50+'
            ELSE 'NA'
        End as age_group,
        EXTRACT(YEAR FROM date_joined) AS post_year,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
    FROM df_all 
    WHERE EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
    GROUP BY age_group, post_year
    Order by age_group, post_year
""")
display(median_follower_per_join_year_age_group)

## Unmount S3 Bucket

If required, the following cell will unmount the S3 bucket from Databricks Filesystem.

In [0]:
if __name__ == "__main__":
    data_loader.unmount_s3_bucket()